In [17]:
from ultralytics import YOLO

In [18]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.utils import to_categorical

import warnings
warnings.filterwarnings('ignore')

In [19]:
import torch

In [20]:
from tqdm import tqdm

In [21]:
import SimpleITK as sitk
import cv2

In [22]:
from IPython.display import display, clear_output
import time

In [23]:
import os
import json

# Описание датасета

Датасет состоит из 489 кейсов (0 - 299, 400 - 588).
Каждый кейс содержит:
1. imaging.nii.gz - трехмерное изображение почек
2. segmentation.nii.gz - трехмерное изображение маски (0 - фон, 1 - почка, 2 - опухоль, 3 - киста)
3. instances - папка, которая содержит все маски по отдельности трижды (зачем трижды?). Они представляют собой маски, где 0 - фон, а 1 - объект данного класса).

Принято решение брать первую аннотацию.

In [69]:
path = "D:\kits23\dataset"
cases1 = list(range(300))
cases2 = list(range(400, 589))
categories = [
    {"id": 0, "name": "kidney", "supercategory": ""},
    {"id": 1, "name": "tumor", "supercategory": ""},
    {"id": 2, "name": "cyst", "supercategory": ""}]

### Методы в помощь

In [25]:
def get_contour_coords(mask):
    """
    Extracts contour coordinates from a segmentation mask.

    Args:
        mask (2d np.array): A binary mask representing the segmentation of an object.

    Returns:
        str: A string containing normalized coordinates of the contours.
    """
    contours, _ = cv2.findContours(mask.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    normalized_coordinates = []
    
    for contour in contours:
        normalized_contour = contour.squeeze() / mask.shape[::-1]
        normalized_coordinates.extend(normalized_contour.flatten().tolist())
    
    coordinates_string = ' '.join(map(str, normalized_coordinates))
    
    return coordinates_string

In [26]:
def get_instances_description(curr_case):
    """
    Get a list of dictionaries describing instances for the given case.

    Args:
        curr_case (int): The current case number.

    Returns:
        list: A list of dictionaries, each describing an instance with keys 'obj_name', 'instance', and 'annotation'.
    """
    curr_path = path + "\\case_00" + str(curr_case).zfill(3) + "\\instances"
    objects = os.listdir(curr_path)
    
    obj_list = []

    for obj in objects:

        obj_name, instance, annotation = obj.split('_')
        _, instance_num = instance.split('-')
        _, annotation_tail = annotation.split('-')
        annotation_num, _, _ = annotation_tail.split('.')

        obj_dict = {}
        obj_dict.update({"obj_name": obj_name})
        obj_dict.update({"instance": int(instance_num)})
        obj_dict.update({"annotation": int(annotation_num)})
        
        obj_list.append(obj_dict)

    return obj_list

In [27]:
def compare_arrays(arr1, arr2):
    """
    Сравнивает два трехмерных массива и находит слои несоответствия.
    
    Аргументы:
        arr1 (np.array): Трехмерный массив 512x512xN, где N - размерность третьего измерения
        arr2 (np.array): Трехмерный массив 512x512xN, где N - размерность третьего измерения
    
    Возвращает:
        list: Список слоёв, который отличается между собой
    """
    different_layers = []
    
    # Проверяем каждый слой массивов на равенство
    for i in range(arr1.shape[2]):
        if not np.array_equal(arr1[:, :, i], arr2[:, :, i]):
            different_layers.append(i)
    
    return different_layers

In [28]:
def get_3d_array_from_path(curr_path):
    curr = sitk.ReadImage(curr_path)
    curr_array = sitk.GetArrayFromImage(curr)
    return curr_array

In [29]:
def get_id(case, layer):
    return str(case).zfill(3) + str(layer).zfill(3)

In [30]:
def save_text_to_file(text, file_path):
    with open(file_path, "w") as file:
        file.write(text)

### Начало сборки масок

In [80]:
for case in range(91, 101):
# for case in cases1: # Раскомментировать здесь, чтобы скачать первую партию изображений
    
    print("Case:", str(case).zfill(3))
    
    # Получение 3D-изображения
    image_path = path + "\\case_00" + str(case).zfill(3) + "\\imaging.nii.gz"
    image_array = get_3d_array_from_path(image_path)
    
    # Получение 3D-маски
    mask_path = path + "\\case_00" + str(case).zfill(3) + "\\segmentation.nii.gz"
    mask_array = get_3d_array_from_path(mask_path)
    
    case_shape = image_array.shape[2]
    
    # Получение словаря, в котором хранятся все маски для каждого класса по отдельности (см. метод выше)
    instances = get_instances_description(case)
    print("instances", instances)
    
    # Получение 3D-масок по экземплярам
    instances_list = []   # Список с 3D-масками экземляров
    instance_cats = []    # Список с соответствующим классом в виде числа
    for instance in instances:
        if instance['annotation'] == 1:
            print("instance", instance)
            
            # Определение id и типа текущего экземляра
            category_name = instance['obj_name']
            for category in categories:
                if category["name"] == category_name:
                    category_id = category["id"]
                    break
            print("category_name", category_name, "category_id", category_id)
            
            # Получение 3D-экземляра
            instance_path = path + "\\case_00" + str(case).zfill(3) + \
            "\\instances\\" + category_name               + \
            "_instance-"    + str(instance['instance'])   + \
            "_annotation-"  + str(instance['annotation']) + ".nii.gz"
            instance_array = get_3d_array_from_path(instance_path)
            
            # Добавление во временные списки
            instance_cats.append(category_id)
            instances_list.append(instance_array)
    
    # Сохранение изображений и лейблов кейса
    images_saved = 0
    for layer in range(case_shape):
        
        image_2d = image_array[:, :, layer]
        mask_2d = mask_array[:, :, layer]
        
        # Если на 2D-маске нет ни одного класса, то изображение не сохраняется
        if np.all(mask_2d == 0):
            continue
        
        contours = ""
        for i, category in enumerate(instance_cats):
            contour = get_contour_coords(instances_list[i][layer])
            if contour:
                contours += str(category) + " " + contour + '\n'
            
             
        if contours:
            # Сохранение лейбла
            save_text_to_file(contours[:-1], r"D:\datasets\my project\train\labels\img_" + get_id(case, layer) + ".txt")
            
            # Сохранение изображения
            plt.imshow(image_2d, cmap='gray')
            plt.axis('off')
            plt.show()
            plt.imsave(r"D:\datasets\my project\train\images\img_" + get_id(case, layer) + ".png", image_2d, cmap='gray') 
    

Case: 091
instances [{'obj_name': 'kidney', 'instance': 1, 'annotation': 1}, {'obj_name': 'kidney', 'instance': 1, 'annotation': 2}, {'obj_name': 'kidney', 'instance': 1, 'annotation': 3}, {'obj_name': 'kidney', 'instance': 2, 'annotation': 1}, {'obj_name': 'kidney', 'instance': 2, 'annotation': 2}, {'obj_name': 'kidney', 'instance': 2, 'annotation': 3}, {'obj_name': 'tumor', 'instance': 1, 'annotation': 1}, {'obj_name': 'tumor', 'instance': 1, 'annotation': 2}, {'obj_name': 'tumor', 'instance': 1, 'annotation': 3}]
instance {'obj_name': 'kidney', 'instance': 1, 'annotation': 1}
category_name kidney category_id 0
instance {'obj_name': 'kidney', 'instance': 2, 'annotation': 1}
category_name kidney category_id 0
instance {'obj_name': 'tumor', 'instance': 1, 'annotation': 1}
category_name tumor category_id 1
Case: 092
instances [{'obj_name': 'cyst', 'instance': 1, 'annotation': 1}, {'obj_name': 'cyst', 'instance': 1, 'annotation': 2}, {'obj_name': 'cyst', 'instance': 1, 'annotation': 3}, 

instance {'obj_name': 'cyst', 'instance': 2, 'annotation': 1}
category_name cyst category_id 2
instance {'obj_name': 'cyst', 'instance': 3, 'annotation': 1}
category_name cyst category_id 2
instance {'obj_name': 'cyst', 'instance': 4, 'annotation': 1}
category_name cyst category_id 2
instance {'obj_name': 'cyst', 'instance': 5, 'annotation': 1}
category_name cyst category_id 2
instance {'obj_name': 'cyst', 'instance': 6, 'annotation': 1}
category_name cyst category_id 2
instance {'obj_name': 'cyst', 'instance': 7, 'annotation': 1}
category_name cyst category_id 2
instance {'obj_name': 'cyst', 'instance': 8, 'annotation': 1}
category_name cyst category_id 2
instance {'obj_name': 'kidney', 'instance': 1, 'annotation': 1}
category_name kidney category_id 0
instance {'obj_name': 'kidney', 'instance': 2, 'annotation': 1}
category_name kidney category_id 0
instance {'obj_name': 'tumor', 'instance': 1, 'annotation': 1}
category_name tumor category_id 1
instance {'obj_name': 'tumor', 'instance

In [81]:
model = YOLO('yolov8n-seg.pt')

results = model.train(data=r"D:\datasets\my project\data.yaml", epochs=2, imgsz=640)

New https://pypi.org/project/ultralytics/8.1.45 available  Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.58  Python-3.9.12 torch-2.0.1+cpu CPU
yolo\engine\trainer: task=segment, mode=train, model=yolov8n-seg.pt, data=D:\datasets\my project\data.yaml, epochs=2, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, box